In [2]:
import os
from PIL import Image
import pandas as pd

In [ ]:
def analyze_images(file_path):
    extensions = ('.jpeg', '.jpg', '.png')
    analysis_data = []

    for class_label in os.listdir(file_path):
        path = os.path.join(file_path, str(class_label))

        if os.path.exists(path):
            files = os.listdir(path)

            image_files = [f for f in files if f.lower().endswith(extensions)]

            for image_file in image_files:
                image_path = os.path.join(path, image_file)

                try:
                    with Image.open(image_path) as img:
                        width, height = img.size
                        mode = img.mode
                        file_size = os.path.getsize(image_path) / 1024 # KB

                        analysis_data.append({
                            'filename': image_file,
                            'class': class_label,
                            'width': width,
                            'height': height,
                            'aspect_ratio': round(width/height, 2),
                            'mode': mode,
                            'file_size_kb': round(file_size, 2),
                            'path': image_path
                        })
                except Exception as e:
                    print(f"Error reading {image_path}: {e}")

    return pd.DataFrame(analysis_data)

In [ ]:
path = 'shared_artifacts/images/hagrid_30k'
# extensions = ('.jpeg', '.jpg', '.png')
extensions = ('.jpg')

for class_label in os.listdir(path):
    pure_label = class_label[10:] # removes 'train_val_' prefix
    label_path = os.path.join(path, class_label)

    if os.path.exists(label_path):
        files = os.listdir(label_path)

        image_files = [f for f in files if f.lower().endswith(extensions)]

        for image in image_files:





# df = analyze_images('shared_artifacts/images/hagrid_30k')

# print(f"Total images: {len(df)}")
# print(f"Number of classes: {df['class'].nunique()}")
# print(f"Min dimensions: {df['width'].min()}x{df['height'].min()}")
# print(f"Max dimensions: {df['width'].max()}x{df['height'].max()}")
# print("\n=== Class Distribution ===")
# class_dist = df['class'].value_counts().sort_index()
# print(class_dist)

Files len: 1763 vs Image files len: 1763
True
Files len: 1783 vs Image files len: 1783
True
Files len: 1735 vs Image files len: 1735
True
Files len: 1805 vs Image files len: 1805
True
Files len: 1732 vs Image files len: 1732
True
Files len: 1811 vs Image files len: 1811
True
Files len: 1750 vs Image files len: 1750
True
Files len: 1778 vs Image files len: 1778
True
Files len: 1770 vs Image files len: 1770
True
Files len: 1769 vs Image files len: 1769
True
Files len: 1742 vs Image files len: 1742
True
Files len: 1736 vs Image files len: 1736
True
Files len: 1748 vs Image files len: 1748
True
Files len: 1803 vs Image files len: 1803
True
Files len: 1751 vs Image files len: 1751
True
Files len: 1737 vs Image files len: 1737
True
Files len: 1855 vs Image files len: 1855
True
Files len: 1765 vs Image files len: 1765
True
